In [ ]:
!pip install statsmodels

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
import time

import random

from statsmodels.tsa.ar_model import AutoReg
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm


In [ ]:
timesteps = 36

In [ ]:
data = pd.read_csv('station_153211-2022-09_2023-01.csv')
print(len(data))
ts = data['available_count']

ts = (ts - ts.min()) / (ts.max() - ts.min())
ts


In [ ]:
plot_pacf(ts, lags=50)
plt.show()

plot_acf(ts, lags=500) 
plt.show()

# Assuming `ts` is your time series data stored as a pandas Series
result = sm.tsa.adfuller(ts)

# Print the test results
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

# Interpretation based on p-value
if result[1] < 0.05:
    print("Strong evidence against the null hypothesis (Ho), reject the null hypothesis. Data has no unit root and is stationary.")
else:
    print("Weak evidence against the null hypothesis, time series has a unit root, indicating it is non-stationary.")



### AR (Auto Regression)

In [ ]:
p = 18

model = AutoReg(ts[:-timesteps], lags=p)
model_fitted = model.fit()

start = len(data) - timesteps
end = start + timesteps 

predictions_AR = model_fitted.predict(start=start, end=end-1, dynamic=False)
true_values = ts[len(data)-timesteps:len(data)]  # Adjust the slicing here
true_values_for_graph = ts[len(data)-timesteps*3:len(data)]  # Adjust the slicing here

plt.plot(predictions_AR, label="Predictions")
plt.plot(true_values_for_graph, label="True Values")
plt.legend()

mse_AR = mean_squared_error(true_values, predictions_AR)
print(mse_AR)


### MA (Moving Average)

In [ ]:
# Fit the MA model (let's assume q=2 based on the ACF plot)
model = ARIMA(ts[:-timesteps], order=(0,0,80))
model_fitted = model.fit()

predictions_MA = model_fitted.predict(start=start, end=end-1)


plt.plot(predictions_MA, label="Predictions")
plt.plot(true_values_for_graph, label="True Values")
plt.legend()

# Calculate MSE at every 10-minute interval, assuming each timestep is 5 minutes
# Start with the first 10 minutes (2 timesteps) and increment by one timestep (5 minutes) each iteration
mse_MA = mean_squared_error(true_values, predictions_MA)
print(mse_MA)


### ARIMA

In [ ]:
# Fit the MA model (let's assume q=2 based on the ACF plot)
model = ARIMA(ts[:-timesteps], order=(p,0,80))  # ARIMA(p=0,d=0,q=2) is equivalent to MA(q=2)
model_fitted = model.fit()

predictions_ARIMA = model_fitted.predict(start=start, end=end-1)

plt.plot(predictions_ARIMA, label="Predictions")
plt.plot(true_values_for_graph, label="True Values")
plt.legend()

# Calculate MSE at every 10-minute interval, assuming each timestep is 5 minutes
# Start with the first 10 minutes (2 timesteps) and increment by one timestep (5 minutes) each iteration
mse_ARIMA = mean_squared_error(true_values, predictions_ARIMA)
print(mse_ARIMA)


### SARIMA

In [ ]:

# Define the SARIMA model parameters
p = 1  # Autoregressive order
d = 0  # Differencing order
q = 1  # Moving average order
P = 1  # Seasonal autoregressive order
D = 0  # Seasonal differencing order
Q = 1  # Seasonal moving average order
s = 90  # Seasonal length in time steps

# Fit the SARIMA model
model = sm.tsa.SARIMAX(ts, order=(p,d,q), seasonal_order=(P,D,Q,s))
model_fitted = model.fit()

# Make forecasts
n_periods = 24  # Number of periods to forecast
forecast = model_fitted.get_forecast(steps=n_periods)
forecast_index = pd.date_range(ts.index[-1], periods=n_periods+1, freq='M')[1:]
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

# Plot the time series and forecasts
plt.figure(figsize=(10, 5))
plt.plot(ts, label='Observed')
plt.plot(forecast_index, forecast_mean, label='Forecast')
plt.fill_between(forecast_index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='grey', alpha=0.3)
plt.legend()
plt.show()

# Print the forecast values
print(forecast_mean)